# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [5]:
# your code here
orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [12]:
# your code here
top20 = orders.nlargest(20,'amount_spent')
top20

customer_spent = orders.groupby(['CustomerID']).sum()[['amount_spent']]
customer_spent

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [18]:
Preferred = customer_spent['amount_spent'].quantile(0.75)
VIP = customer_spent['amount_spent'].quantile(0.95)

In [21]:
# naming all which are above 0,8 and 0,9 with dediceted name in column customer label

customer_spent.loc[customer_spent['amount_spent'] > customer_spent['amount_spent'].quantile(0.80), 'Customer Label'] = 'Preferred'
customer_spent.loc[customer_spent['amount_spent'] > customer_spent['amount_spent'].quantile(0.90), 'Customer Label'] = 'VIP'

# count all VIP and Prefferds
customer_spent['Customer Label'].value_counts()


CustomerID
12346          VIP
12347          VIP
12348    Preferred
12349    Preferred
12350          NaN
           ...    
18280          NaN
18281          NaN
18282          NaN
18283    Preferred
18287    Preferred
Name: Customer Label, Length: 4339, dtype: object

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [25]:
# your code here
VIP_customer = customer_spent[customer_spent['Customer Label'] == 'VIP']['Country'].value_counts()
VIP_customer

United Kingdom     379
Norway              15
Germany             10
France               9
EIRE                 8
Spain                6
Denmark              3
Portugal             2
Channel Islands      1
Japan                1
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [29]:
# your code here
customer_spent.loc[customer_spent['Customer Label'] == 'Preferred', 'Country'] = orders['Country']
Preferred_customer = customer_spent[customer_spent['Customer Label'] == 'Preferred']['Country'].value_counts()
Preferred_customer

United Kingdom     592
Germany             12
France              10
Norway               9
Spain                9
Denmark              5
Japan                5
EIRE                 4
Portugal             4
Channel Islands      1
Name: Country, dtype: int64

In [37]:
vipandpreffered = VIP_customer.add(Preferred_customer, fill_value = 0 ).astype(int)

In [38]:
vipandpreffered

Channel Islands      2
Denmark              8
EIRE                12
France              19
Germany             22
Japan                6
Norway              24
Portugal             6
Spain               15
United Kingdom     971
Name: Country, dtype: int64